In [2]:
#pip install transformers datasets


In [1]:
import pandas as pd

# Example dataset
data = {
    'text': [
        "Select the name and age of all employees who are older than 30.",
        "Get the total sales amount from the sales table.",
        "Find the department with the maximum number of employees."
    ],
    'sql': [
        "SELECT name, age FROM employees WHERE age > 30;",
        "SELECT SUM(sales_amount) FROM sales;",
        "SELECT department FROM employees GROUP BY department ORDER BY COUNT(*) DESC LIMIT 1;"
    ]
}

df = pd.DataFrame(data)
df.to_csv('text_to_sql_dataset.csv', index=False)


PermissionError: [Errno 13] Permission denied: 'text_to_sql_dataset.csv'

In [2]:
from datasets import load_dataset

# Define a preprocessing function
def preprocess_function(examples):
    inputs = examples['text']
    targets = examples['sql']
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=512, truncation=True, padding="max_length")
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# Load dataset
dataset = load_dataset('csv', data_files={'train': 'text_to_sql_dataset.csv'}, split='train')

# Tokenizer
from transformers import AutoTokenizer
model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Preprocess the dataset
encoded_dataset = dataset.map(preprocess_function, batched=True)


In [3]:
from transformers import TrainingArguments

# Define training arguments with output_dir
training_args = TrainingArguments(
    output_dir='./results',  # Directory to save model checkpoints and logs
    per_device_train_batch_size=2,
    num_train_epochs=3,
    logging_dir='./logs',
    logging_steps=10,
)


In [6]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments

# Load dataset
dataset = load_dataset('csv', data_files={'train': 'text_to_sql_dataset.csv'}, split='train')

# Tokenizer
model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define a preprocessing function
def preprocess_function(examples):
    inputs = examples['text']
    targets = examples['sql']
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=512, truncation=True, padding="max_length")
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# Preprocess the dataset
encoded_dataset = dataset.map(preprocess_function, batched=True)

# Load model
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Define training arguments with output_dir
training_args = TrainingArguments(
    output_dir='./results',  # Directory to save model checkpoints and logs
    per_device_train_batch_size=2,
    num_train_epochs=3,
    logging_dir='./logs',
    logging_steps=10,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset,
    tokenizer=tokenizer
)

# Train the model
trainer.train()

# Save the fine-tuned model and tokenizer
model.save_pretrained("./fine-tuned-t5-small")
tokenizer.save_pretrained("./fine-tuned-t5-small")

print("Model and tokenizer saved successfully.")


Step,Training Loss


Model and tokenizer saved successfully.


In [7]:
pip install sentencepiece


   ---------------------------------------- 0.0/991.5 kB ? eta -:--:--
   ------- -------------------------------- 174.1/991.5 kB 5.1 MB/s eta 0:00:01
   ---------------- ----------------------- 399.4/991.5 kB 4.9 MB/s eta 0:00:01
   ---------------- ----------------------- 399.4/991.5 kB 4.9 MB/s eta 0:00:01
   ------------------ --------------------- 460.8/991.5 kB 3.6 MB/s eta 0:00:01
   --------------------- ------------------ 532.5/991.5 kB 2.6 MB/s eta 0:00:01
   --------------------- ------------------ 532.5/991.5 kB 2.6 MB/s eta 0:00:01
   --------------------- ------------------ 532.5/991.5 kB 2.6 MB/s eta 0:00:01
   ----------------------------- ---------- 727.0/991.5 kB 2.2 MB/s eta 0:00:01
   ------------------------------ --------- 747.5/991.5 kB 2.0 MB/s eta 0:00:01
   ------------------------------ --------- 747.5/991.5 kB 2.0 MB/s eta 0:00:01
   ------------------------------ --------- 747.5/991.5 kB 2.0 MB/s eta 0:00:01
   ----------------------------------- ---- 870.4

In [10]:
import os
from transformers import T5ForConditionalGeneration, T5Tokenizer

model_save_path = "./fine-tuned-t5-small"  # Adjust this path as necessary

# Convert to absolute path
model_save_path = os.path.abspath(model_save_path)

print(f"Model save path: {model_save_path}")

# Load the model and tokenizer
model = T5ForConditionalGeneration.from_pretrained(model_save_path)
tokenizer = T5Tokenizer.from_pretrained(model_save_path)

# Define the function to generate SQL
def generate_sql(nl_query):
    input_text = f"translate English to SQL: {nl_query}"
    inputs = tokenizer(input_text, return_tensors="pt")
    outputs = model.generate(**inputs)
    sql_query = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return sql_query

# Test the model
nl_query = "Select the name and age of employees with more than 5 years of experience."
print("Generated SQL:", generate_sql(nl_query))


Model save path: C:\Users\91998\fine-tuned-t5-small


TypeError: not a string